Создайте приложение для эмуляции работа киоска по продаже хот-догов. Приложение должно иметь следующую функциональность:
1. Пользователь может выбрать из трёх стандартных рецептов хот-дога или создать свой рецепт.
2. Пользователь может выбирать добавлять ли майонез,горчицу, кетчуп, топпинги (сладкий лук, халапеньо,чили, соленный огурец и т.д.).
3. Информацию о заказанном хот-доге нужно отображать на экран и сохранять в файл.
4. Если пользователь заказывает от трёх хот-догов нужно предусмотреть скидку.Скидка зависит от количества хот-догов.
5. Расчет может производиться, как наличными, так и картой.
6. Необходимо иметь возможность просмотреть количество проданных хот-догов, выручку, прибыль.
7. Необходимо иметь возможность просмотреть информацию о наличии компонентов для создания хот-дога.
8. Если компоненты для создания хот-догов заканчиваются нужно вывести информационное сообщение о тех компонентах, которые требуется риобрести.
9. Классы приложения должны быть построены с учетом принципов SOLID и паттернов проектирования.

In [1]:
import json
from abc import ABC, abstractmethod

# Класс для представления компонента хот-дога
class HotDog:
    def __init__(self, name, base_price):
        self.name = name
        self.base_price = base_price
        self.toppings = []
        self.sauces = []

    def add_topping(self, topping):
        self.toppings.append(topping)

    def add_sauce(self, sauce):
        self.sauces.append(sauce)

    def calculate_price(self):
        # цена + цена топпингов и соусов
        total_price = self.base_price + len(self.toppings) * 0.6 + len(self.sauces) * 0.4
        return total_price

    def __str__(self):
        return f"{self.name}: {' '.join(self.sauces)} {' '.join(self.toppings)} - ${self.calculate_price():.2f}"


# Класс для управления инвентарем
class Inventory:
    def __init__(self):
        self.components = {
            'майонез': 20,
            'горчица': 20,
            'кетчуп': 20,
            'сладкий лук': 10,
            'халапеньо': 10,
            'чили': 10,
            'соленный огурец': 20,
        }

    def check_availability(self, component):
        return self.components.get(component, 0) > 0

    def use_component(self, component):
        if self.check_availability(component):
            self.components[component] -= 1
        else:
            raise ValueError(f"Недостаточно {component} на складе!")

    def restock(self, component, amount):
        if component in self.components:
            self.components[component] += amount

    def get_low_stock_components(self):
        return [comp for comp, qty in self.components.items() if qty < 3]


# Класс для обработки платежей
class Payment(ABC):
    @abstractmethod
    def process_payment(self, amount):
        pass


class CashPayment(Payment):
    def process_payment(self, amount):
        print(f"Оплачено наличными: ${amount:.2f}")


class CardPayment(Payment):
    def process_payment(self, amount):
        print(f"Оплачено картой: ${amount:.2f}")


# Класс для обработки заказов
class Order:
    def __init__(self):
        self.hot_dogs = []
        self.total_amount = 0

    def add_hot_dog(self, hot_dog):
        self.hot_dogs.append(hot_dog)
        self.total_amount += hot_dog.calculate_price()

    def apply_discount(self):
        if len(self.hot_dogs) >= 3:
            discount = 0.1 * self.total_amount  # 10% скидка
            self.total_amount -= discount

    def generate_report(self):
        report_data = {
            "ordered_hot_dogs": [str(hd) for hd in self.hot_dogs],
            "total_amount": self.total_amount
        }
        with open("order_report.json", "w") as f:
            json.dump(report_data, f, indent=4)


# Основной класс приложения
class HotDogKiosk:
    def __init__(self):
        self.inventory = Inventory()
        self.orders_count = 0
        self.total_revenue = 0.0

    def create_hot_dog(self, name, base_price):
        hot_dog = HotDog(name, base_price)
        return hot_dog

    def take_order(self, order):
        order.apply_discount()
        payment_method = input("Выберите метод оплаты (наличные/карта): ")
        
        if payment_method == "наличные":
            payment = CashPayment()
        elif payment_method == "карта":
            payment = CardPayment()
        else:
            print("Неверный метод оплаты!")
            return
        
        payment.process_payment(order.total_amount)
        # Обновление статистики
        self.orders_count += len(order.hot_dogs)
        self.total_revenue += order.total_amount
        
        # Генерация отчета
        order.generate_report()

    def check_inventory(self):
        low_stock = self.inventory.get_low_stock_components()
        if low_stock:
            print("Нехватка компонентов:", ", ".join(low_stock))
        else:
            print("Все компоненты в наличии!")


kiosk = HotDogKiosk()
    
   
standard_hot_dogs = [
    kiosk.create_hot_dog("Классический", 3.00),
    kiosk.create_hot_dog("Куриный", 4.00)
    ]
    
   
order = Order()
    
while True:
    print("\nДоступные хот-доги:")
    for i, hd in enumerate(standard_hot_dogs):
        print(f"{i + 1}. {hd}")
        
    choice = input("Выберите хот-дог (или 'q' для выхода): ")
        
    if choice.lower() == 'q':
        break
        
    try:
        hot_dog = standard_hot_dogs[int(choice) - 1]
            
            # Добавление соусов и топпингов
        while True:
            topping = input("Добавьте топпинг (или 'done' для завершения): ")
            if topping.lower() == 'done':
                break
            if kiosk.inventory.check_availability(topping):
                hot_dog.add_topping(topping)
                kiosk.inventory.use_component(topping)
            else:
                print(f"Компонент {topping} недоступен.")
        
        while True:
            sauce = input("Добавьте соус (или 'done' для завершения): ")
            if sauce.lower() == 'done':
                break
            if kiosk.inventory.check_availability(sauce):
                hot_dog.add_sauce(sauce)
            kiosk.inventory.use_component(sauce)
        else:
            print(f"Компонент {sauce} недоступен.")
    
        order.add_hot_dog(hot_dog)
    
    except (IndexError, ValueError) as e:
        print("Неверный ввод!")
    
    # Обработка заказа
kiosk.take_order(order)
    
    # Проверка инвентаря
kiosk.check_inventory()
    
print(f"Всего продано хот-догов: {kiosk.orders_count}, Выручка: ${kiosk.total_revenue:.2f}")



Доступные хот-доги:
1. Классический:   - $3.00
2. Куриный:   - $4.00

Доступные хот-доги:
1. Классический:   - $3.00
2. Куриный:   - $4.00
Оплачено картой: $3.00
Все компоненты в наличии!
Всего продано хот-догов: 1, Выручка: $3.00
